In [1]:
import pandas as pd
import numpy as np

In [4]:
listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)

In [7]:
listing_details.shape[0]

32461

In [6]:
listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18534 entries, 0 to 18533
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        18534 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           18534 non-null  object 
 2   Xã/Phường/Thị trấn                    18321 non-null  object 
 3   Đường phố                             15355 non-null  object 
 4   Chi tiết                              18534 non-null  object 
 5   Nguồn thông tin                       18534 non-null  object 
 6   Tình trạng giao dịch                  18534 non-null  object 
 7   Thời điểm giao dịch/rao bán           18524 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 17012 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    18534 non-null  object 
 11  Số tầng công tr

In [10]:
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18534 entries, 0 to 18533
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        18534 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           18534 non-null  object 
 2   Xã/Phường/Thị trấn                    18321 non-null  object 
 3   Đường phố                             15355 non-null  object 
 4   Chi tiết                              18534 non-null  object 
 5   url                                   18534 non-null  object 
 6   Tình trạng giao dịch                  18534 non-null  object 
 7   Thời điểm giao dịch/rao bán           18524 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 17012 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    18534 non-null  object 
 11  Số tầng công tr

In [12]:
cleaned = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18536 entries, 0 to 18535
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        18536 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           18536 non-null  object 
 2   Xã/Phường/Thị trấn                    18323 non-null  object 
 3   Đường phố                             15357 non-null  object 
 4   Chi tiết                              18536 non-null  object 
 5   url                                   18536 non-null  object 
 6   Tình trạng giao dịch                  18536 non-null  object 
 7   Thời điểm giao dịch/rao bán           18526 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 17014 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    18536 non-null  object 
 11  Số tầng công tr

In [35]:
import numpy as np

def correct_phuong(row):
    xaphuong = row['Xã/Phường/Thị trấn']
    short_add = row['short_address']
    if type(xaphuong) == float or type(short_add) == float:
        return 0
    if xaphuong in short_add:
        return 1
    return 0

cleaned['correct_phuong'] = cleaned.apply(correct_phuong, axis=1)
incorrect_phuong = cleaned[cleaned['correct_phuong'] == 0]

In [42]:
pd.set_option('display.max_columns', None)

incorrect_phuong[['Xã/Phường/Thị trấn', 'short_address']]

,Xã/Phường/Thị trấn,short_address
176,NaN,Nhà 2 mặt tiền hẻm xe tải thông hướng Âu Cơ Tr...
305,NaN,Nhà 3 tầng Nguyễn Viết Xuân Hòa Minh Liên Chiể...
360,Phường Yên Nghĩa,"Yên Nghĩa, Hà Đông, Hà Nội"
368,NaN,"Bạch Mai, Bạch Mai, Hai Bà Trưng, Hà Nội"
397,Phường Mỹ Đình 2,"Mỹ Đình 2, Nam Từ Liêm, Hà Nội"
470,Phường Hiệp Bình Chánh,"Hiệp Bình Chánh, Thủ Đức, Hồ Chí Minh"
474,NaN,"Nguyễn Sơn, Bồ Đề, Long Biên, Hà Nội"
547,Phường Linh Đông,"Linh Đông, Thủ Đức, Hồ Chí Minh"
564,NaN,"Đặng Thùy Trâm, 13, Bình Thạnh, Hồ Chí Minh"
619,NaN,"Thạch Cầu, Long Biên, Long Biên, Hà Nội"


- 176: NaN --> Xã Yên Nghĩa
- 305: NaN --> 